In [1]:
import sys
import cobra
#If you have CPLEX, uncomment this
#import cplex
import cobrakbase
#Put the path to ModelSEEDpy on your machine here
sys.path.append("/Users/chenry/code/ModelSEEDpy")
#import modelseedpy.fbapkg
from modelseedpy import GapfillingPkg, KBaseMediaPkg
from modelseedpy import FBAHelper, MSBuilder


cobrakbase 0.2.7


In [2]:
kbase_api = cobrakbase.KBaseAPI()
model = kbase_api.get_from_ws("test_model",18528)
#If you have CPLEX, uncomment this
#model.solver = 'optlang-cplex'
template = kbase_api.get_from_ws("GramNegModelTemplateV3","NewKBaseModelTemplates")
media = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")
model = MSBuilder.gapfill_model(model,"bio1",template,media)

kmp = KBaseMediaPkg(model)
kmp.build_package(media)
sol=model.optimize()
model.summary()